# Amazon Bedrock InvokeModel

In [ ]:
import boto3
import json


# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client("bedrock-runtime",
                      aws_access_key_id="",
                      aws_secret_access_key="", 
                      region_name="us-east-1")

# Set the model ID, e.g., Claude 3 Haiku.
model_id = "anthropic.claude-3-haiku-20240307-v1:0"


# Define the prompt for the model.
prompt = "tell me about the nepal"


# Format the request payload using the model's native structure.
native_request = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 512,
    "temperature": 0.5,
    "messages": [
         {
            "role": "user",
            "content": [{"type": "text", "text": f"{prompt} my name is miraj"}],
        }
    ],
}

request = json.dumps(native_request)

response = client.invoke_model(modelId=model_id,
                                body=request)

model_response = json.loads(response["body"].read())

# Extract and print the response text.
response_text = model_response["content"][0]["text"]

print(response_text)



Here are some key facts about Nepal for you, Miraj:

- Nepal is a landlocked country located in South Asia, sandwiched between India and China. Its capital and largest city is Kathmandu.

- Nepal is known for its stunning Himalayan mountain ranges, including Mount Everest, the highest peak in the world. The Himalayas cover about 15% of Nepal's total land area.

- Nepal has a diverse population of over 29 million people, with over 100 different ethnic groups and 123 spoken languages. The two main languages are Nepali and Newari.

- The majority of Nepalis practice Hinduism, with a significant Buddhist minority. Nepal is the only official Hindu state in the world.

- Nepal has a rich cultural heritage, with ancient temples, monasteries, and traditional festivals throughout the country. Some of the most famous sites include Boudhanath Stupa and Swayambhunath Stupa in Kathmandu.

- The Nepalese economy is largely based on agriculture, with major exports including carpets, textiles, and agr

# Amazon Bedrock Converse

In [ ]:
# Use the Conversation API to send a text message to Anthropic Claude.

import boto3
from botocore.exceptions import ClientError

# Create a Bedrock Runtime client in the AWS Region you want to use.
client = boto3.client("bedrock-runtime",
                      aws_access_key_id="",
                      aws_secret_access_key="", 
                      region_name="us-east-1")

# Set the model ID, e.g., Claude 3 Haiku.
model_id = "anthropic.claude-3-haiku-20240307-v1:0"

# Start a conversation with the user message.
my_info = "my name is miraj deep. i am 40 years old"
conversation = [
    {
        "role": "user",
        "content": [{"text": my_info}],
    },
    {
        "role": "assistant",
        "content": [{"text": "hi miraj! how can i help you"}],
    },
    {
        "role": "user",
        "content": [{"text": "what is the capital city of nepal?"}],
    },
    {
        "role": "assistant",
        "content": [{"text": "the capital city of nepal is kathmandu"}],
    },
    {
        "role": "user",
        "content": [{"text": "dont give me my name i need only my age. and dont give any preamble"}],
    }
    ,
    {
        "role": "assistant",
        "content": [{"text": "40"}],
    },
    {
        "role": "user",
        "content": [{"text": "what is the first question i asked you can you give me that"}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    response = client.converse(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )
    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)




what is the capital city of nepal?


In [ ]:

import boto3
from botocore.exceptions import ClientError


def generate_conversation(bedrock_client,
                          model_id,
                          system_prompts,
                          messages):


    inference_config = {"temperature": 0.5}
    additional_model_fields = {"top_k": 200}

    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages,
        system=system_prompts,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields
    )

    return response

def main():


    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

    system_prompts = [{
        "text": "You are an app that creates playlists for a radio station that plays rock and pop music. Only return song names and the artist."
    }]
    
    message_1 = {
        "role": "user",
        "content": [{"text": "Create a list of 3 pop songs."}]
    }
    
    message_2 = {
        "role": "user",
        "content": [{"text": "Make sure the songs are by artists from the United Kingdom."}]
    }
    
    messages = []

    try:
        # ❗ Dummy credentials for demonstration ONLY
        bedrock_client = boto3.client("bedrock-runtime",
                      aws_access_key_id="",
                      aws_secret_access_key="",
                      region_name="us-east-1")

        messages.append(message_1)
        response = generate_conversation(bedrock_client, model_id, system_prompts, messages)

        output_message = response['output']['message']
        messages.append(output_message)

        messages.append(message_2)
        response = generate_conversation(bedrock_client, model_id, system_prompts, messages)

        output_message = response['output']['message']
        messages.append(output_message)

        for message in messages:
            print(f"Role: {message['role']}")
            for content in message['content']:
                print(f"Text: {content['text']}")
            print()

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occurred: {message}")

    else:
        print(f"Finished generating text with model {model_id}.")

if __name__ == "__main__":
    main()


Role: user
Text: Create a list of 3 pop songs.

Role: assistant
Text: Here are 3 pop songs:

1. "Blinding Lights" by The Weeknd
2. "Levitating" by Dua Lipa
3. "Watermelon Sugar" by Harry Styles

Role: user
Text: Make sure the songs are by artists from the United Kingdom.

Role: assistant
Text: Here are 3 pop songs by artists from the United Kingdom:

1. "Shape of You" by Ed Sheeran
2. "Someone You Loved" by Lewis Capaldi
3. "Vossi Bop" by Stormzy

Finished generating text with model anthropic.claude-3-sonnet-20240229-v1:0.
